In [83]:
#spacy english en_core_web_trf

#Librerias a usar

from bert_serving.client import BertClient
import spacy
from collections import Counter
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/alvaromontero/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/alvaromontero/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [63]:
#Cargar libros
Book_list=["/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/divine_comedy.txt",'/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Oddysey.txt'
           ,'/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Illiad.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Great_Gatsby.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Tom_Sawyer.txt','/mnt/d/Repositorios/Tecnologias_de_lenguaje_natural/P2/Practica3/Wuthering_Heights.txt']
#Book_list=[r"C:\Users\diavl\OneDrive\Escritorio\Repositorios\Tecnologias_de_lenguaje_natural\P2\Practica3\divine_comedy.txt",r"C:\Users\diavl\OneDrive\Escritorio\Repositorios\Tecnologias_de_lenguaje_natural\P2\Practica3\Oddysey.txt",r"C:\Users\diavl\OneDrive\Escritorio\Repositorios\Tecnologias_de_lenguaje_natural\P2\Practica3\Illiad.txt",
#                r"C:\Users\diavl\OneDrive\Escritorio\Repositorios\Tecnologias_de_lenguaje_natural\P2\Practica3\Great_Gatsby.txt",r"C:\Users\diavl\OneDrive\Escritorio\Repositorios\Tecnologias_de_lenguaje_natural\P2\Practica3\Tom_Sawyer.txt",r"C:\Users\diavl\OneDrive\Escritorio\Repositorios\Tecnologias_de_lenguaje_natural\P2\Practica3\Wuthering_Heights.txt"]

Books_raw_file=[]
for i in range(len(Book_list)):
    with open(Book_list[i],'r',encoding='utf-8')as f:
        file=f.read()
        Books_raw_file.append(file)
        
        


PREPROCESAMIENTO


In [64]:
#Funciones

def eliminar_etiquetas(text):
    pattern = re.compile(r'<[^>]+>|\([^)]+\)|--[^\-]+--')
    doc = pattern.sub('', text)
    return doc

# Removemos apostrofes para quedarnos con palabras como cant couldnt o dont
def remover_apostrofes(text):
    pattern = r"\\?'"
    doc = re.sub(pattern, '', text)
    return doc
# Nos quedamos con solo alfabeto eliminando caracteres especiales    
def remover_especiales(text):
    pattern = r"[^a-z\s]"
    doc = re.sub(pattern, ' ', text)
    return doc
# Quitamos dobles espacios para tener texto espaciado solamente por un espacio por palabra
def dobles_espacios(text):
    pattern = r'\s+'
    doc = re.sub(pattern, ' ', text)
    return doc
def remove_stopwords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

In [65]:
#Vectorizacion tfidf
def vectorizar_tfidf(corpus):
    # TF-IDF
    vectorizer = TfidfVectorizer(ngram_range=(4,4))
    tfidf_matrix = vectorizer.fit_transform(corpus)
    
    # palabras únicas
    feature_names_tfidf = vectorizer.get_feature_names_out()
    vocabulary = vectorizer.get_feature_names_out()
    print("Tamaño de Palabras únicas:", len(feature_names_tfidf))
    return tfidf_matrix,vocabulary

In [66]:
#Quitar inicio de texto
deleted_lines_list=[134,336,1997,32,459,7]

for i,book in enumerate(Books_raw_file):
    lines = book.splitlines()  # Split the book text into lines
    modified_lines = lines[deleted_lines_list[i]:]  # Keep lines starting from the specified index
    modified_book = "\n".join(modified_lines)  # Join the remaining lines back into a string
    Books_raw_file[i]=modified_book


In [67]:
for i,book in enumerate(Books_raw_file):
    Books_raw_file[i]=book.lower()
    
print(Books_raw_file[0][:200])

canto i


in the midway of this our mortal life,
i found me in a gloomy wood, astray
gone from the path direct: and e’en to tell
it were no easy task, how savage wild
that forest, how robust and rough


Eliminacion de caracteres no importantes


In [68]:
from copy import deepcopy
books_clean_files=[]
books_clean_stops_file=[]
for i,book in enumerate(Books_raw_file):
    Text_aux=eliminar_etiquetas(book)
    Text_aux=remover_apostrofes(Text_aux)
    Text_aux=dobles_espacios(Text_aux)
    Text_aux=remover_especiales(Text_aux)
    
    books_clean_stops_file.append(deepcopy(Text_aux))#Por si el modelo no necesita quitar stopwords
    
    Text_aux=remove_stopwords(Text_aux)
    books_clean_files.append(Text_aux)#Texto sin stopwords
    print(Text_aux[:45]+'\n')

canto midway mortal life found gloomy wood as

book gods council minerva visit ithaca challe

iliad book argument contention achilles agame

younger vulnerable years father gave advice t

chapter tom answer tom answer gone boy wonder

chapter returned visit landlord solitary neig



Extracción de oraciones clave

1) TF-IDF

In [86]:
#TF-IDF NLTK

for book in books_clean_files:#Iteramos sobre los libros para obtener lo mas representativo de cada texto
    tokens=word_tokenize(book)
    #postags=pos_tag(tokens)
    stemmer=PorterStemmer()
    stemmed_tokens=[stemmer.stem(token) for token in tokens]
    
    
    vectorizer = TfidfVectorizer(ngram_range=(4,4))
    tfidf_matrix = vectorizer.fit_transform([stemmed_tokens])
    feature_names_tfidf = vectorizer.get_feature_names_out()
    vocabulary = vectorizer.get_feature_names_out()
    print("Tamaño de Palabras únicas:", len(feature_names_tfidf))
    frecuencia_total = Counter()
    for ngrama in vocabulary:
    # Obtiene el índice de la palabra en el vocabulario
        indice = vectorizer.vocabulary_[ngrama]
        # Suma las frecuencias de esa palabra en todas las frases
        frecuencia_total[ngrama] = tfidf_matrix[:, indice].sum()
    frases_mas_frecuentes = frecuencia_total.most_common(5)
    for i in range(5):
        print(f"Frase: {frases_mas_frecuentes[i][0]}, Frecuencia: {frases_mas_frecuentes[i][1]}")#El decimal es que tanto contribuyen al "Corpus"




ValueError: empty vocabulary; perhaps the documents only contain stop words

KeyboardInterrupt: 